In [ ]:
import  torch, os
import  numpy as np
from    MiniImagenet import MiniImagenet
import  scipy.stats
from    torch.utils.data import DataLoader
from    torch.optim import lr_scheduler
import  random, sys, pickle
import  argparse

from    torch import nn
from    torch import optim
from    torch.nn import functional as F

from    learner import Learner
# from    copy import deepcopy
from attack import PGD


from StandardTrans import Transfer
from LoadDataST import ImagenetMini



def mean_confidence_interval(accs, confidence=0.95):
    n = accs.shape[0]
    m, se = np.mean(accs), scipy.stats.sem(accs)
    h = se * scipy.stats.t._ppf((1 + confidence) / 2, n - 1)
    return m, h


def main():

#     torch.manual_seed(222)
#     torch.cuda.manual_seed_all(222)
#     np.random.seed(222)

    print(args)

    config = [
        ('conv2d', [32, 3, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 2, 0]),
        ('conv2d', [32, 32, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 2, 0]),
        ('conv2d', [32, 32, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 2, 0]),
        ('conv2d', [32, 32, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 1, 0]),
        ('flatten', []),
        ('linear', [64, 32 * 5 * 5])
    ]

    device = torch.device('cuda:4')
    
    start_epoch = 0
    start_step = 0
    filename = 'standardtrainadv.pt'
    transferlearn = Learner(config, args.imgc, args.imgsz).to(device)
#     trans_optim = optim.Adam(transferlearn.parameters(), lr=args.meta_lr)
    criterion = torch.nn.CrossEntropyLoss()
    
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_step = checkpoint['step']
        transferlearn.load_state_dict(checkpoint['state_dict'])
        #maml = maml.to(device)
        print("=> loaded checkpoint '{}' (step {})"
                  .format(filename, checkpoint['step']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    trans_optim = torch.optim.SGD(transferlearn.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(trans_optim, milestones=[30, 85, 108], gamma=0.1)
    #trans_optim = Lamb(transferlearn.parameters(), lr=0.01, weight_decay=1e-4, betas=(.9, .999), adam=False)
    tmp = filter(lambda x: x.requires_grad, transferlearn.parameters())
    num = sum(map(lambda x: np.prod(x.shape), tmp))
    print(transferlearn)
    print(type(transferlearn))
    print('Total trainable tensors:', num)

    # batchsz here means total episode number
#     mini = MiniImagenet('../../../dataset/', mode='train', n_way=args.n_way, k_shot=5,
#                         k_query=5,
#                         batchsz=10000, resize=args.imgsz)
    mini = ImagenetMini('../../../dataset/', mode='train',
                        batchsz=10000, resize=args.imgsz)
    mini_test = MiniImagenet('../../../dataset/', mode='test', n_way=5, k_shot=1,
                             k_query=args.k_qry,
                             batchsz=100, resize=args.imgsz)
    
    


#     for epoch in range(args.epoch//10000):
#         # fetch meta_batchsz num of episode each time
#         db = DataLoader(mini, args.task_num, shuffle=True, num_workers=0, pin_memory=True)
    at = PGD(eps=2 / 255.0, sigma=2 / 255.0, nb_iter=10)

    for step in range(start_step, 30000):


        x, y = mini.loading_data.get_next_batch(64, multiple_passes=True, reshuffle_after_pass=True)





        x_sq = x.to(device)
        y_sq = y.to(device)

        querysz = x_sq.size(0)
        
        
        
        trans_optim.zero_grad()


        adv_inp = at.attack(transferlearn, transferlearn.parameters(), x_sq, y_sq)
        trans_optim.zero_grad()
        transferlearn.train()
        logits = transferlearn(adv_inp, vars=None, bn_training=True)
        loss = criterion(logits, y_sq)

#         acc = torch_accuracy(pred, label, (1,))
#         advacc = acc[0].item()
#         advloss = loss.item()
        (loss * 1.0).backward()
        trans_optim.step()
    
        trans_optim.zero_grad()




        logits = transferlearn(x_sq, vars=None, bn_training=True)
        loss = criterion(logits, y_sq)
        with torch.no_grad():
            pred = F.softmax(logits, dim=1).argmax(dim=1)
            correct = torch.eq(pred, y_sq).sum().item()


        trans_optim.zero_grad()
        loss.backward()
        trans_optim.step()
        lr_scheduler.step()



        accs = np.array(correct) / (querysz)

        if step % 30 == 0:                   
            print('step:', step, '\ttraining acc:', accs)
#                 print('step:', step, '\ttraining acc_adv:', accs_adv)
            state = {'step': step, 'state_dict': transferlearn.state_dict()}
            torch.save(state, 'standardtrainadv.pt')

        if step % 500 == 0:  # evaluation

            db_test = DataLoader(mini_test, 1, shuffle=True, num_workers=0, pin_memory=True)
            accs_all_test = []
            accsadv_all_test = []
            accsadvpr_all_test = []
            transfertst = Transfer(args, config).to(device)






            for x_spt, y_spt, x_qry, y_qry in db_test:
                x_spt, y_spt, x_qry, y_qry = x_spt.squeeze(0).to(device), y_spt.squeeze(0).to(device), \
                                             x_qry.squeeze(0).to(device), y_qry.squeeze(0).to(device)

                accs, accs_adv, accs_adv_prior = transfertst.finetunning(x_spt, y_spt, x_qry, y_qry, transferlearn)
                accs_all_test.append(accs)
                accsadv_all_test.append(accs_adv)
                accsadvpr_all_test.append(accs_adv_prior)

            # [b, update_step+1]
            accs = np.array(accs_all_test).mean(axis=0).astype(np.float16)
            accs_adv = np.array(accsadv_all_test).mean(axis=0).astype(np.float16)
            accs_adv_prior = np.array(accsadvpr_all_test).mean(axis=0).astype(np.float16)
            print('Test acc:', accs)
            print('Test acc_adv:', accs_adv)
            print('Test acc_adv_prior:', accs_adv_prior)


if __name__ == '__main__':

    argparser = argparse.ArgumentParser()
    argparser.add_argument('--epoch', type=int, help='epoch number', default=100000)
    argparser.add_argument('--n_way', type=int, help='n way', default=5)
    argparser.add_argument('--k_spt', type=int, help='k shot for support set', default=1)
    argparser.add_argument('--k_qry', type=int, help='k shot for query set', default=15)
    argparser.add_argument('--imgsz', type=int, help='imgsz', default=84)
    argparser.add_argument('--imgc', type=int, help='imgc', default=3)
    argparser.add_argument('--task_num', type=int, help='meta batch size, namely task num', default=1)
    argparser.add_argument('--meta_lr', type=float, help='meta-level outer learning rate', default=1e-3)
    argparser.add_argument('--update_lr', type=float, help='task-level inner update learning rate', default=0.01)
    argparser.add_argument('--update_step', type=int, help='task-level inner update steps', default=5)
    argparser.add_argument('--update_step_test', type=int, help='update steps for finetunning', default=10)
    
    #argparser.add_argument('--fast', action="store_true", help='whether to use fgsm')

    args = argparser.parse_args(args=[])

    main()